In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input/ssl-50/ssl50'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/8a/83/6674d8f1a7cabb6a6a50defeb629c04e9b8b24798b5962c1075fed2c40a7/mediapipe-0.10.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 31.1 MB/s eta 0:00:00


In [3]:
import os

# specify the name of the folder you want to create
folder_name = "skeleton_videos"
# folder_name = "lsa-skeleton_videos"

# create the folder if it does not already exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created!")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'skeleton_videos' created!


In [4]:
import csv
csv_file = open('skeleton_videos/skeleton_locations.csv', 'w', newline='')
# csv_file = open('ssl-skeleton_videos/lsa-tr-skeleton_locations.csv', 'w', newline='')

writer = csv.writer(csv_file)
writer.writerow(['file_name','nose', 'left_eye_inner','left_eye', 
                         'left_eye_outer', 'right_eye_inner', 
                         'right_eye', 'right_eye_outer' ,'left_ear',
                         'right_ear', 'mouth_left', 'mouth_right',
                         'left_elbow', 'right_elbow','right_shoulder', 'left_shoulder', 'left_pinky_tip', 'right_pinky_tip',
                        'left_wrist','left_thumb_cmc','left_thumb_mcp','left_thumb_ip',
    'left_thumb_tip','left_index_finger_mcp','left_index_finger_pip','left_index_finger_dip',
    'left_index_finger_tip','left_middle_finger_mcp','left_middle_finger_pip',
    'left_middle_finger_dip','left_middle_finger_tip','left_ring_finger_mcp',
    'left_ring_finger_pip','left_ring_finger_dip','left_ring_finger_tip',
    'left_pinky_mcp','left_pinky_pip','left_pinky_dip',
    'right_wrist','right_thumb_cmc','right_thumb_mcp',
    'right_thumb_ip','right_thumb_tip','right_index_finger_mcp',
    'right_index_finger_pip','right_index_finger_dip','right_index_finger_tip',
    'right_middle_finger_mcp','right_middle_finger_pip',
    'right_middle_finger_dip','right_middle_finger_tip',
    'right_ring_finger_mcp','right_ring_finger_pip','right_ring_finger_dip',
    'right_ring_finger_tip','right_pinky_mcp','right_pinky_pip',
    'right_pinky_dip'])

1000

In [5]:
def normalize_coordinates(coordinates, frame_width, frame_height):
    normalized_coordinates = []
    for x, y in coordinates:
        normalized_x = x / float(frame_width)
        normalized_y = y / float(frame_height)
        normalized_coordinates.append((normalized_x, normalized_y))
    return normalized_coordinates

In [6]:
import mediapipe as mp
import cv2
import csv

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

E0000 00:00:1703134436.248473      20 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1703134436.248541      20 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1703134436.248559      20 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1703134436.248577      20 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1703134436.248594      20 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1703134436.24

In [7]:

def extract_skeleton_locations(video_file):
    cap = cv2.VideoCapture(video_file)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results_pose = pose.process(frame)
        results_hands = hands.process(frame)
        annotated_image = frame.copy()
#         if results_pose.pose_landmarks:
#             mp_drawing.draw_landmarks(annotated_image, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)
#         if results_hands.multi_hand_landmarks:
#             for hand_landmarks in results_hands.multi_hand_landmarks:
#                 mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        file_name = video_file.split('/')[-1]
        # Body
        right_shoulder_locations = []
        left_shoulder_locations = []
        nose_locations = []    
        left_eye_inner_locations = []
        left_eye_locations = []
        left_eye_outer_locations = []
        right_eye_inner_locations = []
        right_eye_locations = []
        right_eye_outer_locations = []
        left_ear_locations = []
        right_ear_locations = []
        mouth_left_locations = []
        mouth_right_locations = []
        left_elbow_locations = []
        right_elbow_locations = []
        #hand
        left_pinky_tip_locations = []
        right_pinky_tip_locations = []
        left_wrist_locations = []
        left_thumb_cmc_locations = []
        left_thumb_mcp_locations = []
        left_thumb_ip_locations = []
        left_thumb_tip_locations = []
        left_index_finger_mcp_locations = []
        left_index_finger_pip_locations = []
        left_index_finger_dip_locations = []
        left_index_finger_tip_locations = []
        left_middle_finger_mcp_locations = []
        left_middle_finger_pip_locations = []
        left_middle_finger_dip_locations = []
        left_middle_finger_tip_locations = []
        left_ring_finger_mcp_locations = []
        left_ring_finger_pip_locations = []
        left_ring_finger_dip_locations = []
        left_ring_finger_tip_locations = []
        left_pinky_mcp_locations = []
        left_pinky_pip_locations = []
        left_pinky_dip_locations = []
        
        right_wrist_locations = []
        right_thumb_cmc_locations = []
        right_thumb_mcp_locations = []
        right_thumb_ip_locations = []
        right_thumb_tip_locations = []
        right_index_finger_mcp_locations = []
        right_index_finger_pip_locations = []
        right_index_finger_dip_locations = []
        right_index_finger_tip_locations = []
        right_middle_finger_mcp_locations = []
        right_middle_finger_pip_locations = []
        right_middle_finger_dip_locations = []
        right_middle_finger_tip_locations = []
        right_ring_finger_mcp_locations = []
        right_ring_finger_pip_locations = []
        right_ring_finger_dip_locations = []
        right_ring_finger_tip_locations = []
        right_pinky_mcp_locations = []
        right_pinky_pip_locations = []
        right_pinky_dip_locations = []
        #hand both
        wrist_locations = []
        thumb_cmc_locations = []
        thumb_mcp_locations = []
        thumb_ip_locations = []
        thumb_tip_locations = []
        index_finger_mcp_locations = []
        index_finger_pip_locations = []
        index_finger_dip_locations = []
        index_finger_tip_locations = []
        middle_finger_mcp_locations = []
        middle_finger_pip_locations = []
        middle_finger_dip_locations = []
        middle_finger_tip_locations = []
        ring_finger_mcp_locations = []
        ring_finger_pip_locations = []
        ring_finger_dip_locations = []
        ring_finger_tip_locations = []
        pinky_tip_locations = []
        pinky_mcp_locations = []
        pinky_pip_locations = []
        pinky_dip_locations = []
        
        if results_pose.pose_landmarks:
            for i, landmark in enumerate(results_pose.pose_landmarks.landmark):
                if landmark.visibility < 0.05:
                    landmark_px = (0, 0)
                else:
                    landmark_px = (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0]))
                if not landmark_px:
                    landmark_px = (0, 0)
                # Normalize the landmark coordinates
                frame_width = frame.shape[1]
                frame_height = frame.shape[0]
                normalized_landmark_px = normalize_coordinates([landmark_px], frame_width, frame_height)[0]
                landmark_px = normalized_landmark_px
                if i == mp_pose.PoseLandmark.RIGHT_SHOULDER.value:
                    right_shoulder_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_SHOULDER.value:
                    left_shoulder_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.NOSE.value:
                    nose_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_EYE_INNER.value:
                    left_eye_inner_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_EYE.value:
                    left_eye_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_EYE_OUTER.value:
                    left_eye_outer_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_EYE_INNER.value:
                    right_eye_inner_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_EYE.value:
                    right_eye_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_EYE_OUTER.value:
                    right_eye_outer_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_EAR.value:
                    left_ear_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_EAR.value:
                    right_ear_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.MOUTH_LEFT.value:
                    mouth_left_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.MOUTH_RIGHT.value:
                    mouth_right_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.LEFT_ELBOW.value:
                    left_elbow_locations.append(landmark_px)
                elif i == mp_pose.PoseLandmark.RIGHT_ELBOW.value:
                    right_elbow_locations.append(landmark_px)
        if results_hands.multi_hand_landmarks:
            for hand_landmarks in results_hands.multi_hand_landmarks:
                for i, landmark in enumerate(hand_landmarks.landmark):
                    landmark_px = (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0]))
                    frame_width = frame.shape[1]
                    frame_height = frame.shape[0]
                    normalized_landmark_px = normalize_coordinates([landmark_px], frame_width, frame_height)[0]
                    landmark_px = normalized_landmark_px
                    if i == mp_hands.HandLandmark.PINKY_TIP.value:
                        pinky_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.WRIST.value:
                        wrist_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.THUMB_CMC.value:
                        thumb_cmc_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.THUMB_MCP.value:
                        thumb_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.THUMB_IP.value:
                        thumb_ip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.THUMB_TIP.value:
                        thumb_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.INDEX_FINGER_MCP.value:
                        index_finger_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.INDEX_FINGER_PIP.value:
                        index_finger_pip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.INDEX_FINGER_DIP.value:
                        index_finger_dip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.INDEX_FINGER_TIP.value:
                        index_finger_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.MIDDLE_FINGER_MCP.value:
                        middle_finger_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value:
                        middle_finger_pip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.MIDDLE_FINGER_DIP.value:
                        middle_finger_dip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value:
                        middle_finger_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.RING_FINGER_MCP.value:
                        ring_finger_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.RING_FINGER_PIP.value:
                        ring_finger_pip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.RING_FINGER_DIP.value:
                        ring_finger_dip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.RING_FINGER_TIP.value:
                        ring_finger_tip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.PINKY_MCP.value:
                        pinky_mcp_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.PINKY_PIP.value:
                        pinky_pip_locations.append(landmark_px)
                    elif i == mp_hands.HandLandmark.PINKY_DIP.value:
                        pinky_dip_locations.append(landmark_px)
        if len(pinky_tip_locations) > 0:
            right_pinky_tip_locations = [pinky_tip_locations[0]]
        else:
            right_pinky_tip_locations = [(0,0)]
        if len(pinky_tip_locations) > 1:
            left_pinky_tip_locations = [pinky_tip_locations[1]]
        else:
            left_pinky_tip_locations = [(0,0)]
            #################################
        if len(wrist_locations) > 0:
            right_wrist_locations = [wrist_locations[0]]
        else:
            right_wrist_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_wrist_locations = [wrist_locations[1]]
        else:
            left_wrist_locations = [(0,0)]
            ################################
        if len(wrist_locations) > 0:
            right_thumb_cmc_locations = [thumb_cmc_locations[0]]
        else:
            right_thumb_cmc_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_thumb_cmc_locations = [thumb_cmc_locations[1]]
        else:
            left_thumb_cmc_locations = [(0,0)]
           ################################
        if len(wrist_locations) > 0:
            right_thumb_mcp_locations = [thumb_mcp_locations[0]]
        else:
            right_thumb_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_thumb_mcp_locations = [thumb_mcp_locations[1]]
        else:
            left_thumb_mcp_locations = [(0,0)]
           ################################
        if len(wrist_locations) > 0:
            right_thumb_ip_locations = [thumb_ip_locations[0]]
        else:
            right_thumb_ip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_thumb_ip_locations = [thumb_ip_locations[1]]
        else:
            left_thumb_ip_locations = [(0,0)]
           ################################
        if len(wrist_locations) > 0:
            right_thumb_tip_locations = [thumb_tip_locations[0]]
        else:
            right_thumb_tip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_thumb_tip_locations = [thumb_tip_locations[1]]
        else:
            left_thumb_tip_locations = [(0,0)]
          ################################
        if len(wrist_locations) > 0:
            right_index_finger_mcp_locations = [index_finger_mcp_locations[0]]
        else:
            right_index_finger_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_index_finger_mcp_locations = [index_finger_mcp_locations[1]]
        else:
            left_index_finger_mcp_locations = [(0,0)]
         ################################
        if len(wrist_locations) > 0:
            right_index_finger_pip_locations = [index_finger_pip_locations[0]]
        else:
            right_index_finger_pip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_index_finger_pip_locations = [index_finger_pip_locations[1]]
        else:
            left_index_finger_pip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_index_finger_dip_locations = [index_finger_dip_locations[0]]
        else:
            right_index_finger_dip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_index_finger_dip_locations = [index_finger_dip_locations[1]]
        else:
            left_index_finger_dip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_index_finger_tip_locations = [index_finger_tip_locations[0]]
        else:
            right_index_finger_tip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_index_finger_tip_locations = [index_finger_tip_locations[1]]
        else:
            left_index_finger_tip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_middle_finger_mcp_locations = [middle_finger_mcp_locations[0]]
        else:
            right_middle_finger_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_middle_finger_mcp_locations = [middle_finger_mcp_locations[1]]
        else:
            left_middle_finger_mcp_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_middle_finger_pip_locations = [middle_finger_pip_locations[0]]
        else:
            right_middle_finger_pip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_middle_finger_pip_locations = [middle_finger_pip_locations[1]]
        else:
            left_middle_finger_pip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_middle_finger_dip_locations = [middle_finger_dip_locations[0]]
        else:
            right_middle_finger_dip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_middle_finger_dip_locations = [middle_finger_dip_locations[1]]
        else:
            left_middle_finger_dip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_middle_finger_tip_locations = [middle_finger_tip_locations[0]]
        else:
            right_middle_finger_tip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_middle_finger_tip_locations = [middle_finger_tip_locations[1]]
        else:
            left_middle_finger_tip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_ring_finger_mcp_locations = [ring_finger_mcp_locations[0]]
        else:
            right_ring_finger_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_ring_finger_mcp_locations = [ring_finger_mcp_locations[1]]
        else:
            left_ring_finger_mcp_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_ring_finger_pip_locations = [ring_finger_pip_locations[0]]
        else:
            right_ring_finger_pip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_ring_finger_pip_locations = [ring_finger_pip_locations[1]]
        else:
            left_ring_finger_pip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_ring_finger_dip_locations = [ring_finger_dip_locations[0]]
        else:
            right_ring_finger_dip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_ring_finger_dip_locations = [ring_finger_dip_locations[1]]
        else:
            left_ring_finger_dip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_ring_finger_tip_locations = [ring_finger_tip_locations[0]]
        else:
            right_ring_finger_tip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_ring_finger_tip_locations = [ring_finger_tip_locations[1]]
        else:
            left_ring_finger_tip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_pinky_mcp_locations = [pinky_mcp_locations[0]]
        else:
            right_pinky_mcp_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_pinky_mcp_locations = [pinky_mcp_locations[1]]
        else:
            left_pinky_mcp_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_pinky_pip_locations = [pinky_pip_locations[0]]
        else:
            right_pinky_pip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_pinky_pip_locations = [pinky_pip_locations[1]]
        else:
            left_pinky_pip_locations = [(0,0)]
        ################################
        if len(wrist_locations) > 0:
            right_pinky_dip_locations = [pinky_dip_locations[0]]
        else:
            right_pinky_dip_locations = [(0,0)]
        if len(wrist_locations) > 1:
            left_pinky_dip_locations = [pinky_dip_locations[1]]
        else:
            left_pinky_dip_locations = [(0,0)]
        nose_locations = nose_locations if nose_locations else [(0,0)]
        left_eye_inner_locations = left_eye_inner_locations if left_eye_inner_locations else [(0,0)]
        left_eye_locations = left_eye_locations if left_eye_locations else [(0,0)]
        left_eye_outer_locations = left_eye_outer_locations if left_eye_outer_locations else [(0,0)]
        right_eye_inner_locations = right_eye_inner_locations if right_eye_inner_locations else [(0,0)]
        right_eye_locations = right_eye_locations if right_eye_locations else [(0,0)]
        right_eye_outer_locations = right_eye_outer_locations if right_eye_outer_locations else [(0,0)]
        left_ear_locations = left_ear_locations if left_ear_locations else [(0,0)]
        right_ear_locations = right_ear_locations if right_ear_locations else [(0,0)]
        mouth_left_locations = mouth_left_locations if mouth_left_locations else [(0,0)]
        mouth_right_locations = mouth_right_locations if mouth_right_locations else [(0,0)]
        left_elbow_locations = left_elbow_locations if left_elbow_locations else [(0,0)]
        right_elbow_locations = right_elbow_locations if right_elbow_locations else [(0,0)]
        right_shoulder_locations = right_shoulder_locations if right_shoulder_locations else [(0,0)]
        left_shoulder_locations = left_shoulder_locations if left_shoulder_locations else [(0,0)]

        writer.writerow([file_name,nose_locations, left_eye_inner_locations,left_eye_locations, 
                         left_eye_outer_locations, right_eye_inner_locations, 
                         right_eye_locations, right_eye_outer_locations ,left_ear_locations,
                         right_ear_locations, mouth_left_locations, mouth_right_locations,
                         left_elbow_locations, right_elbow_locations ,right_shoulder_locations, left_shoulder_locations, left_pinky_tip_locations, right_pinky_tip_locations,
                        left_wrist_locations,left_thumb_cmc_locations,left_thumb_mcp_locations,left_thumb_ip_locations,
    left_thumb_tip_locations,left_index_finger_mcp_locations,left_index_finger_pip_locations,left_index_finger_dip_locations,
    left_index_finger_tip_locations,left_middle_finger_mcp_locations,left_middle_finger_pip_locations,
    left_middle_finger_dip_locations,left_middle_finger_tip_locations,left_ring_finger_mcp_locations,
    left_ring_finger_pip_locations,left_ring_finger_dip_locations,left_ring_finger_tip_locations,
    left_pinky_mcp_locations,left_pinky_pip_locations,left_pinky_dip_locations,
    right_wrist_locations,right_thumb_cmc_locations,right_thumb_mcp_locations,
    right_thumb_ip_locations,right_thumb_tip_locations,right_index_finger_mcp_locations,
    right_index_finger_pip_locations,right_index_finger_dip_locations,right_index_finger_tip_locations,
    right_middle_finger_mcp_locations,right_middle_finger_pip_locations,
    right_middle_finger_dip_locations,right_middle_finger_tip_locations,
    right_ring_finger_mcp_locations,right_ring_finger_pip_locations,right_ring_finger_dip_locations,
    right_ring_finger_tip_locations,right_pinky_mcp_locations,right_pinky_pip_locations,
    right_pinky_dip_locations])
#     csv_file.close()

In [8]:
# extract_skeleton_locations('/kaggle/input/testset/trimmed_001.mp4')

In [9]:
for dirname, _, filenames in os.walk('/kaggle/input/ssl-50/ssl50'):
# for dirname, _, filenames in os.walk('/kaggle/input/lsa64-videos/Sign_Classes_Videos_Split/Sign_Classes_Videos_Split/train'):
    count = 1
    for filename in filenames:
        path = str(os.path.join(dirname, filename))
        extract_skeleton_locations(path)        
        print(count)
        count += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153


[h264 @ 0x5b8bba60eb40] cabac decode of qscale diff failed at 117 60
[h264 @ 0x5b8bba60eb40] error while decoding MB 117 60, bytestream 9255


154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403


[h264 @ 0x5b8bbab220c0] cabac decode of qscale diff failed at 40 67
[h264 @ 0x5b8bbab220c0] error while decoding MB 40 67, bytestream 495


866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092

# create a flattened matrix 

In [10]:
#For all cells in a row - seperate X -Y values 
import csv

# Specify the output file paths
# input_path = '/kaggle/working/lsa-skeleton_videos/lsa-tr-skeleton_locations.csv'
# output_path = '/kaggle/working/lsa-skeleton_videos/lsa-tr-output_XY_file.csv'
input_path = '/kaggle/working/skeleton_videos/skeleton_locations.csv'
output_path = '/kaggle/working/skeleton_videos/output_XY_file.csv'

# Open the input CSV file and the output CSV file
with open(input_path, 'r', encoding='utf-8') as input_file, open(output_path, 'w', newline='') as output_file:

    # Create a CSV reader object and a CSV writer object
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)
    
    next(reader)

    # Iterate over each row in the input CSV file
    for row in reader:
        
        transformed_row = []
        
        file_name = row[0]
        
        for cell in row[1:]:

            # Get the value from the current cell
            cell_value = cell
        
            if cell_value == 'file_name':
                continue

            numbers = cell_value.strip("[]()").split(",")

            result = tuple(float(x.strip("()[] ")) for x in numbers)

            # Extract the two values from the cell value
            value1, value2 = result
            
            transformed_row.append(value1)
            transformed_row.append(value2)
        
        transformed_row.insert(0, file_name)
        
        # Write the two values to a new row in the output CSV file
        writer.writerow(transformed_row)

In [11]:
#Get all skeleton locations into a single cell

import csv

# Specify the input and output file paths
input_path = '/kaggle/working/skeleton_videos/output_XY_file.csv'
output_path = '/kaggle/working/skeleton_videos/output_SingleCell_file.csv'
# input_path = '/kaggle/working/lsa-skeleton_videos/lsa-tr-output_XY_file.csv'
# output_path = '/kaggle/working/lsa-skeleton_videos/lsa-tr-output_SingleCell_file.csv'

# Open the input CSV file and the output CSV file
with open(input_path, 'r') as input_file, open(output_path, 'w', newline='') as output_file:

    # Create a CSV reader object and a CSV writer object
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)
    
    next(reader)

    # Iterate over each row in the input CSV file
    for row in reader:
        
        transformed_row = []
        
        file_name = row[0]
        
        for cell in row[1:]:

            # Get the value from the current cell
            cell_value = cell
        
            if cell_value == 'file_name':
                continue

            transformed_row.append(float(cell_value))
        
        # Write the two values to a new row in the output CSV file
        writer.writerow([file_name, transformed_row])

In [12]:
import csv

# Load the CSV file
with open('/kaggle/working/skeleton_videos/output_SingleCell_file.csv', 'r') as f:
# with open('/kaggle/working/lsa-skeleton_videos/lsa-tr-output_SingleCell_file.csv', 'r') as f:

    reader = csv.reader(f)
    
    data = {}
    for row in reader:
        filename = row[0]
        skeleton = row[1]
        if filename in data:
            data[filename].append(skeleton)
        else:
            data[filename] = [skeleton]
#     print(data)

# Write the grouped data to a new CSV file
with open('/kaggle/working/ssl-matrix_file.csv', 'w', newline='') as f:
# with open('/kaggle/working/lsa-tr-matrix_file.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for filename, skeletons in data.items():
        
        number_str = filename[:3]  # extract the first three characters
        number = int(number_str)  # convert to integer
        label = number - 1  # subtract one
        
        skeletons.insert(0,label)
        writer.writerow(skeletons)